<a href="https://colab.research.google.com/github/aureliodeboa/Deep-Learning/blob/main/Transfer_Learning_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Learning using MNIST data
To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [4]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [5]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

11490434/11490434 [==============================] - 0s 0us/step


In [8]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [9]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [10]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [11]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [12]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 14s 11ms/step - loss: 1.6090 - accuracy: 0.2105 - val_loss: 1.5931 - val_accuracy: 0.2938
Epoch 2/5
230/230 [==============================] - 2s 7ms/step - loss: 1.5835 - accuracy: 0.2734 - val_loss: 1.5648 - val_accuracy: 0.4571
Epoch 3/5
230/230 [==============================] - 2s 7ms/step - loss: 1.5586 - accuracy: 0.3458 - val_loss: 1.5342 - val_accuracy: 0.6091
Epoch 4/5
230/230 [==============================] - 2s 7ms/step - loss: 1.5290 - accuracy: 0.4168 - val_loss: 1.5006 - val_accuracy: 0.6997
Epoch 5/5
230/230 [==============================] - 2s 7ms/step - loss: 1.4979 - accuracy: 0.4770 - val_loss: 1.4630 - val_accuracy: 0.7406
Training time: 0:00:21.502242
Test score: 1.4629862308502197
Test accuracy: 0.740588366985321


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [13]:
# Freeze only the
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [15]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 2s 7ms/step - loss: 1.5602 - accuracy: 0.3392 - val_loss: 1.5280 - val_accuracy: 0.5188
Epoch 2/5
240/240 [==============================] - 1s 6ms/step - loss: 1.5147 - accuracy: 0.4284 - val_loss: 1.4798 - val_accuracy: 0.6225
Epoch 3/5
240/240 [==============================] - 2s 7ms/step - loss: 1.4727 - accuracy: 0.5020 - val_loss: 1.4320 - val_accuracy: 0.6929
Epoch 4/5
240/240 [==============================] - 1s 5ms/step - loss: 1.4279 - accuracy: 0.5709 - val_loss: 1.3846 - val_accuracy: 0.7472
Epoch 5/5
240/240 [==============================] - 1s 5ms/step - loss: 1.3869 - accuracy: 0.6151 - val_loss: 1.3391 - val_accuracy: 0.7881
Training time: 0:00:10.964040
Test score: 1.3390671014785767
Test accuracy: 0.7880910634994507


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster.

## Exercise
### To do:
- Now write code to reverse this training process.  That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

Train model 0-4

In [16]:
# un-Freeze only the
for l in feature_layers:
    l.trainable = True

In [17]:
# We create our model by combining the two sets of layers as follows
model2 = Sequential(feature_layers + classification_layers)

In [18]:
# Let's take a look
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                      

In [19]:
# Now, let's train our model on the digits 0,1,2,3,4

train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 3s 8ms/step - loss: 1.3385 - accuracy: 0.6669 - val_loss: 1.2798 - val_accuracy: 0.8221
Epoch 2/5
240/240 [==============================] - 2s 8ms/step - loss: 1.2812 - accuracy: 0.7067 - val_loss: 1.2125 - val_accuracy: 0.8494
Epoch 3/5
240/240 [==============================] - 2s 7ms/step - loss: 1.2188 - accuracy: 0.7397 - val_loss: 1.1380 - val_accuracy: 0.8698
Epoch 4/5
240/240 [==============================] - 2s 7ms/step - loss: 1.1463 - accuracy: 0.7717 - val_loss: 1.0564 - val_accuracy: 0.8860
Epoch 5/5
240/240 [==============================] - 2s 7ms/step - loss: 1.0710 - accuracy: 0.7940 - val_loss: 0.9704 - val_accuracy: 0.9013
Training time: 0:00:10.346405
Test score: 0.9703669548034668
Test accuracy: 0.9013426899909973


In [20]:
# Freeze only the
for l in feature_layers:
    l.trainable = False

In [21]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                      

In [22]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 2s 6ms/step - loss: 1.3980 - accuracy: 0.4559 - val_loss: 1.3291 - val_accuracy: 0.5676
Epoch 2/5
230/230 [==============================] - 1s 5ms/step - loss: 1.3460 - accuracy: 0.5036 - val_loss: 1.2748 - val_accuracy: 0.6237
Epoch 3/5
230/230 [==============================] - 1s 5ms/step - loss: 1.2983 - accuracy: 0.5446 - val_loss: 1.2247 - val_accuracy: 0.6733
Epoch 4/5
230/230 [==============================] - 2s 7ms/step - loss: 1.2544 - accuracy: 0.5789 - val_loss: 1.1780 - val_accuracy: 0.7079
Epoch 5/5
230/230 [==============================] - 1s 6ms/step - loss: 1.2114 - accuracy: 0.6168 - val_loss: 1.1343 - val_accuracy: 0.7387
Training time: 0:00:07.608026
Test score: 1.1342872381210327
Test accuracy: 0.7387368679046631
